In [1]:
import numpy as np
import pandas as pd
import re
import ast
import matplotlib.pyplot as plt
#!pip install wordcloud
import math

In [2]:
#path = input('Enter path to master dataset')
path = 'combinedMasterLatest.csv'

In [3]:
df = pd.read_csv (path) 

In [4]:
df.columns

Index(['Unnamed: 0', 'USN', '10thPercentage', '12thPercentage', 'AwardsNLP',
       'Branch', 'CGPA', 'CTC', 'CoCurricularActivities', 'CompanyNameNLP',
       'CoreCourseGrade', 'CoreCourseName', 'CourseGrade', 'CourseName',
       'ElectiveCourseGrade', 'ElectiveCourseName', 'EmploymentType',
       'ExternalCertificatesDomain', 'ExternalCertificatesKey',
       'GeneralSkills', 'GitHubLink', 'InternshipCompany',
       'InternshipProjectDomain', 'LanguagesNLP', 'LinkedInLink',
       'MinorAttended', 'Name', 'NoofInternships', 'NoofProjects',
       'NumberOfOffers', 'OtherDetails', 'ProgLanguages',
       'ProjectDetailDomain', 'PublicationNLP', 'ResearchDomain',
       'ScholarshipsNLP', 'Stipend(K)', 'TierLevel', 'VolunteeringWork',
       'WorkshopsDomain', 'WorkshopsOrg', 'SoftwareTools', 'ProgLanguagesNLP'],
      dtype='object')

In [5]:
df = df.rename({'CompanyNameNLP':'CompanyName_New'}, axis =1)

In [6]:
df_courses = df[['CompanyName_New','ElectiveCourseName','ElectiveCourseGrade']]


In [7]:
df_companyDet = df[['CompanyName_New','CTC','TierLevel']]
df_companyDet = df_companyDet[df_companyDet['CompanyName_New'].notna()]
df_companyDet = df_companyDet.fillna(0)
#df_companyDet['CTC_avg'] = df_companyDet.groupby(['CompanyName_New'])['CTC'].apply(lambda x : x.mean())
#df_companyDet.groupby(['CompanyName_New'])['TierLevel']
df_ctc = df_companyDet.groupby('CompanyName_New')['CTC'].apply(lambda x : x.astype(float)[x!=0].mean()).to_frame()
df_tier = df_companyDet.groupby('CompanyName_New')['TierLevel'].apply(lambda x :x.astype(float)[x!=0].mean()).to_frame()
df_ctc.fillna(0,inplace =True)
df_tier.fillna(0,inplace = True)
df_tier = df_tier.apply(np.floor)

In [8]:
df_company_det = pd.concat([df_ctc, df_tier], axis=1)
df_company_det

,CTC,TierLevel
CompanyName_New,,
Accenture,5.013889,2.0
Adobe,12.430000,1.0
Akamai Technologies,16.481765,1.0
Amadeus Software Labs,9.535714,1.0
Amagi,15.000000,1.0
Amazon,7.000000,1.0
Analog Devices,20.500000,1.0
App Dynamics,39.200000,1.0
Cisco Systems,22.629565,1.0


In [9]:
df_courses.CompanyName_New.isnull().value_counts()

False    380
True     118
Name: CompanyName_New, dtype: int64

In [10]:
df_courses = df_courses[df_courses['CompanyName_New'].notna()]

In [11]:
#---------------------------------------------regularizing subjects -----------------------------------------------------------
# done'unified_engineering','unified_engineering_(jedi)',
# done'engineering_mechanics','engineering_mechanics-dynamics',
# done 'introduction_to_data_science','introduction_to_data_sciences',
# done  __ to _
# done system -> systems
# done enginering -> engineering
# done dbms -> data_base_management_systems
# done -  to -
# done & to and
# done metal_casting_and_welding', 'metal_casting_and_welding_processes',
# done 'power_electronics', 'power_electronics_and_its_applications_to_converters',
# done 'web_technologies-i','web_technologies-ii', 'web_technologies_i', 'web_technologies_ii', 'web_technology-_i',
# 'wireless_communication','wireless_network_communications'
# done processors to processor 
def regularize_courses_grades(l,lg):
    lg = regularize_grades(lg)
    l_new = []
    lg_new = []
    for i in range(0,len(l)):
        l[i] = l[i].replace('-',' ')
        l[i] = re.sub(' +',' ',l[i])
        l[i] = l[i].replace('&','and').replace('systems','system').replace('sciences','science').replace('processors','processor').replace('technology','technologies').replace('enginering','engineering').replace('1','i').replace('2','ii')
        l[i]= l[i].strip().replace(' ','_')
        if(l[i].count('laboratory')>0):
            continue
        if(lg[i] == -1):
            continue
        if(l[i]=='unified_engineering_(jedi)'):
            l[i]= 'unified_engineering'
        if(l[i]=='wireless_communication'):
            l[i]= 'wireless_network_communications'
        if(l[i] == 'engineering_mechanics_dynamics'):
            l[i] = 'engineering_mechanics'
        if(l[i].count('dbms')>0):
            l[i] = l[i].replace('dbms','data_base_management_systems')
        if(l[i].count('metal_casting_and_welding')>0):
            l[i] = 'metal_casting_and_welding'
        if(l[i].count('power_electronics')>0):
            l[i] = 'power_electronics'
        if(l[i].count('mini_project')>0):
            l[i] = 'mini_project'
        if(l[i].count('vlsi')>0):
            l[i] = 'vlsi'
        if(l[i].count('project_work')>0):
            l[i] = 'project_work'
            
        # in case of repetitions, take highest grade   
        if(l[i] in l_new):
            if(int(lg[i])>int(lg_new[l_new.index(l[i])])):
                lg_new[l_new.index(l[i])] = str(lg[i])
            continue       
        l_new.append(l[i])
        lg_new.append(str(lg[i]))
   
    #res = [i +'@@'+ j for i, j in zip(l_unique, lg_new)] 
    lc = '##'.join(l_new)
    lg = '##'.join(lg_new)
    res = lc+'@@'+lg
    return res
        

In [12]:
#---------------------------------------------regularizing grades -----------------------------------------------------------

def regularize_grades(l):
    d = {'S':6,'A':5,'B':4, 'C':3, 'D':2, 'E':1, 'F':0, 'AP': -1}
    for i in range(len(l)):
        l[i] = l[i].replace('"','')
        l[i] = d[l[i]]
    return l
        

In [13]:
#df_courses['CourseGrade_New'] = df_courses.apply(lambda row : regularize_grades( list(ast.literal_eval(row['CourseGrade']))), axis = 1) 

In [15]:
df_courses['CourseDetails'] = df_courses.apply(lambda row : regularize_courses_grades( list(ast.literal_eval(row['ElectiveCourseName'])), list(ast.literal_eval(row['ElectiveCourseGrade']))), axis = 1) 

In [16]:
df_courses[['CourseName_New','CourseGrade_New']] = df_courses.CourseDetails.str.split("@@",expand=True) 

In [17]:
df_courses

,CompanyName_New,ElectiveCourseName,ElectiveCourseGrade,CourseDetails,CourseName_New,CourseGrade_New
0,Cisco Systems,"['Advanced Algorithms', 'Artificial Intelligen...","['A', 'B', 'A', 'B']",Advanced_Algorithms##Artificial_Intelligence##...,Advanced_Algorithms##Artificial_Intelligence##...,5##4##5##4
1,App Dynamics,"['Big Data', 'Data Analytics', 'Topics in Deep...","['S', 'S', 'S', 'S']",Big_Data##Data_Analytics##Topics_in_Deep_Learn...,Big_Data##Data_Analytics##Topics_in_Deep_Learn...,6##6##6##6
2,Wipro,[],[],@@,,
3,Societe Generale,"['Big Data', 'Data Analytics', 'Topics in Deep...","['S', 'S', 'A', 'A']",Big_Data##Data_Analytics##Topics_in_Deep_Learn...,Big_Data##Data_Analytics##Topics_in_Deep_Learn...,6##6##5##5
4,Morgan Stanley,"['Big Data', 'Data Analytics', 'Topics in Deep...","['S', 'S', 'A', 'A']",Big_Data##Data_Analytics##Topics_in_Deep_Learn...,Big_Data##Data_Analytics##Topics_in_Deep_Learn...,6##6##5##5
...,...,...,...,...,...,...
486,Walmart Labs,['Advanced Algorithms'],['S'],Advanced_Algorithms@@6,Advanced_Algorithms,6
487,Amazon,['Advanced Algorithms'],['S'],Advanced_Algorithms@@6,Advanced_Algorithms,6
489,LAM Research,"['Big Data', 'Knowledge Management', 'Programm...","['S', 'S', 'A']",Big_Data##Knowledge_Management##Programming_wi...,Big_Data##Knowledge_Management##Programming_wi...,6##6##5
493,Cisco Systems,"['Big Data', 'Digital Image Processing']","['A', 'A']",Big_Data##Digital_Image_Processing@@5##5,Big_Data##Digital_Image_Processing,5##5


In [18]:
#getting distinct list of all courses, after cleaning
course = df_courses.CourseName_New.apply(lambda x: pd.value_counts(x.split("##"))).sum(axis = 0).to_frame()
course_all = course.index.values.tolist()

#print(len(sorted(course_all)))
#sorted(course_all)

In [19]:
#getting distinct list of all courses, after cleaning
grades = df_courses.CourseGrade_New.apply(lambda x: pd.value_counts(x.split("##"))).sum(axis = 0).to_frame()
grades_all = grades.index.values.tolist()
#print(len(sorted(course_all)))
#sorted(course_all)
grades_all

['5', '4', '6', '', '3', '2', '1']

In [20]:
# removing courses that appear in more than 75% of the data, since they will mostly be core courses
course = df_courses.CourseName_New.apply(lambda x: pd.value_counts(x.split("##"))).sum(axis = 0).lt(1*df_courses.shape[0]).to_frame()
course_distinct = course[course[0] == True]
course_distinct = course_distinct.index.values.tolist()
#print(len(course_distinct), course_distinct)

In [21]:
# function to find occurence of specific courses and creating new column for each unqiue course. Fill with 1/0 based on if course was taken or not
def keyword_matching_course(keywords,df):
    df = df.join(pd.DataFrame(0, columns=keywords, index=df.index))
    df['CourseNameKey'] = ''
    #print(df.shape)
    for key in keywords:
        #print(key)
        for i in range(0, len(df)):
            if(key in df.iloc[i]['CourseName_New'].split('##')):
                df.iloc[i, df.columns.get_loc('CourseNameKey')] =df.iloc[i, df.columns.get_loc('CourseNameKey')]+ key+'##'
                df.iloc[i, df.columns.get_loc(key)] = 1
    return df

In [22]:
df_course_details = df_courses
df_courses = keyword_matching_course(course_distinct,df_courses)

In [23]:
df_courses

,CompanyName_New,ElectiveCourseName,ElectiveCourseGrade,CourseDetails,CourseName_New,CourseGrade_New,Artificial_Intelligence,Topics_in_Deep_Learning,Digital_Image_Processing,Advanced_Algorithms,...,Programming_with_C++,Professional_Skills_for_Software_Engineers_I,Multimedia_Computing,Computer_Graphics_and_Visualization,Programming_with_Java,Knowledge_management,Content_Management,Social_Network_Analytics,Introduction_to_Software_Testing,CourseNameKey
0,Cisco Systems,"['Advanced Algorithms', 'Artificial Intelligen...","['A', 'B', 'A', 'B']",Advanced_Algorithms##Artificial_Intelligence##...,Advanced_Algorithms##Artificial_Intelligence##...,5##4##5##4,1,1,1,1,...,0,0,0,0,0,0,0,0,0,Artificial_Intelligence##Topics_in_Deep_Learni...
1,App Dynamics,"['Big Data', 'Data Analytics', 'Topics in Deep...","['S', 'S', 'S', 'S']",Big_Data##Data_Analytics##Topics_in_Deep_Learn...,Big_Data##Data_Analytics##Topics_in_Deep_Learn...,6##6##6##6,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Topics_in_Deep_Learning##Big_Data##Knowledge_M...
2,Wipro,[],[],@@,,,0,0,0,0,...,0,0,0,0,0,0,0,0,0,##
3,Societe Generale,"['Big Data', 'Data Analytics', 'Topics in Deep...","['S', 'S', 'A', 'A']",Big_Data##Data_Analytics##Topics_in_Deep_Learn...,Big_Data##Data_Analytics##Topics_in_Deep_Learn...,6##6##5##5,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Topics_in_Deep_Learning##Big_Data##Data_Analyt...
4,Morgan Stanley,"['Big Data', 'Data Analytics', 'Topics in Deep...","['S', 'S', 'A', 'A']",Big_Data##Data_Analytics##Topics_in_Deep_Learn...,Big_Data##Data_Analytics##Topics_in_Deep_Learn...,6##6##5##5,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Topics_in_Deep_Learning##Big_Data##Data_Analyt...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,Walmart Labs,['Advanced Algorithms'],['S'],Advanced_Algorithms@@6,Advanced_Algorithms,6,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Advanced_Algorithms##
487,Amazon,['Advanced Algorithms'],['S'],Advanced_Algorithms@@6,Advanced_Algorithms,6,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Advanced_Algorithms##
489,LAM Research,"['Big Data', 'Knowledge Management', 'Programm...","['S', 'S', 'A']",Big_Data##Knowledge_Management##Programming_wi...,Big_Data##Knowledge_Management##Programming_wi...,6##6##5,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Big_Data##Knowledge_Management##Programming_wi...
493,Cisco Systems,"['Big Data', 'Digital Image Processing']","['A', 'A']",Big_Data##Digital_Image_Processing@@5##5,Big_Data##Digital_Image_Processing,5##5,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Digital_Image_Processing##Big_Data##


In [24]:
# getting total number of entries per company, and total sum of each course for each company
dfCourseCount = df_courses.groupby(['CompanyName_New'])[course_distinct].apply(lambda x : x.astype(int).sum())
dfCompanyCount = df_courses.groupby(['CompanyName_New'])['ElectiveCourseName'].count().to_frame().rename(columns = {'ElectiveCourseName': 'Total'}, inplace = False)

In [25]:
result = pd.concat([dfCompanyCount, dfCourseCount], axis=1)

In [26]:
# getting a rating by calculating, total number of students placed at a specific company who took that course/ total number of students placed at that company
result = result[course_distinct].div(result.Total, axis=0)
#result = result.reset_index(level=['CompanyName_New'])

In [27]:
originalRatingsData = result.stack().reset_index().rename(columns={'level_0':'CompanyName','level_1':'ElectiveCourseName', 0:'Rating'})

In [30]:
# function to find occurence of specific courses and creating new column for each unqiue course. Fill with grades
def course_rating_grade(keywords,df):
    df = df.join(pd.DataFrame(-1, columns=keywords, index=df.index))
    #print(sorted(list(df.columns)))
    for key in keywords:
        for i in range(0, len(df)):
            if(key in df.iloc[i]['CourseName_New'].split('##')):
                temp_name = df.iloc[i]['CourseName_New'].split('##')
                temp_grade = df.iloc[i]['CourseGrade_New'].split('##')
                if(key!=''):
                    df.iloc[i, df.columns.get_loc(key)] = int(temp_grade[temp_name.index(key)])
    return df

In [31]:
result_grade = course_rating_grade(course_all, df_course_details)

In [32]:
result_grade

,CompanyName_New,ElectiveCourseName,ElectiveCourseGrade,CourseDetails,CourseName_New,CourseGrade_New,Artificial_Intelligence,Topics_in_Deep_Learning,Digital_Image_Processing,Advanced_Algorithms,...,Parallel_Computing,Programming_with_C++,Professional_Skills_for_Software_Engineers_I,Multimedia_Computing,Computer_Graphics_and_Visualization,Programming_with_Java,Knowledge_management,Content_Management,Social_Network_Analytics,Introduction_to_Software_Testing
0,Cisco Systems,"['Advanced Algorithms', 'Artificial Intelligen...","['A', 'B', 'A', 'B']",Advanced_Algorithms##Artificial_Intelligence##...,Advanced_Algorithms##Artificial_Intelligence##...,5##4##5##4,4,5,4,5,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,App Dynamics,"['Big Data', 'Data Analytics', 'Topics in Deep...","['S', 'S', 'S', 'S']",Big_Data##Data_Analytics##Topics_in_Deep_Learn...,Big_Data##Data_Analytics##Topics_in_Deep_Learn...,6##6##6##6,-1,6,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,Wipro,[],[],@@,,,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,Societe Generale,"['Big Data', 'Data Analytics', 'Topics in Deep...","['S', 'S', 'A', 'A']",Big_Data##Data_Analytics##Topics_in_Deep_Learn...,Big_Data##Data_Analytics##Topics_in_Deep_Learn...,6##6##5##5,-1,5,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,Morgan Stanley,"['Big Data', 'Data Analytics', 'Topics in Deep...","['S', 'S', 'A', 'A']",Big_Data##Data_Analytics##Topics_in_Deep_Learn...,Big_Data##Data_Analytics##Topics_in_Deep_Learn...,6##6##5##5,-1,5,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,Walmart Labs,['Advanced Algorithms'],['S'],Advanced_Algorithms@@6,Advanced_Algorithms,6,-1,-1,-1,6,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
487,Amazon,['Advanced Algorithms'],['S'],Advanced_Algorithms@@6,Advanced_Algorithms,6,-1,-1,-1,6,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
489,LAM Research,"['Big Data', 'Knowledge Management', 'Programm...","['S', 'S', 'A']",Big_Data##Knowledge_Management##Programming_wi...,Big_Data##Knowledge_Management##Programming_wi...,6##6##5,-1,-1,-1,-1,...,-1,5,-1,-1,-1,-1,-1,-1,-1,-1
493,Cisco Systems,"['Big Data', 'Digital Image Processing']","['A', 'A']",Big_Data##Digital_Image_Processing@@5##5,Big_Data##Digital_Image_Processing,5##5,-1,-1,5,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [33]:
# getting total number of entries per company, and total sum of each course for each company
dfCourseGradeAvg = result_grade.groupby(['CompanyName_New'])[course_all].apply(lambda x : x.astype(int)[x!=-1].mean())
#dfCompanyCount = df_courses.groupby(['CompanyName_New'])['CourseName'].count().to_frame().rename(columns = {'CourseName': 'Total'}, inplace = False)

In [34]:
dfCourseGradeAvg.fillna(0,inplace = True)

In [35]:
originalGradeRatingsData = dfCourseGradeAvg.stack().reset_index().rename(columns={'level_0':'CompanyName','level_1':'CourseName', 0:'GradeRating'})